# Analyse de Sentiments

Regardez la vidéo pour plus de détails.

## Code final
Voici le code final:

In [ ]:
# Cette section est uniquement importable dans le backtester
from quantopian.algorithm import attach_pipeline, pipeline_output

# Importations générales de pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import AverageDollarVolume


# Utiliser l'échantillon gratuit dans votre algo de pipeline
from quantopian.pipeline.data.sentdex import sentiment_free as sentdex

# pour calculer l'impact
from quantopian.pipeline.factors import CustomFactor
import numpy as np


def initialize(context):
    # Planifier notre fonction de rééquilibrage au début de chaque semaine.
    schedule_function(my_rebalance, date_rules.every_day())

    
    attach_pipeline(make_pipeline(), "pipeline")

# Calcul du sentiment m sur la longueur de la fenêtre
class AvgSentiment(CustomFactor):
    def compute(self, today, assets, out, impact):
        np.mean(impact, axis=0, out=out)
    
def make_pipeline():

    # Éliminez les actions à un penny et les titres peu liquides.
    dollar_volume = AverageDollarVolume(window_length=30)
    is_liquid = dollar_volume.top(500)
    
    # Calcul du sentiment moyen
    avg_sentiment = AvgSentiment(inputs=[sentdex.sentiment_signal], window_length=10)

    return Pipeline(columns={
            'sentiment':avg_sentiment
        }, screen=is_liquid)
            

    
def before_trading_start(context, data):
    portfo = pipeline_output('pipeline')
    context.something = portfo['sentiment'] 
    
    # Actions avec un sentiment > 0
    context.longs = portfo[portfo['sentiment'] > 0].index.tolist()
    
    # Actions avec un sentiment < 0
    context.shorts = portfo[portfo['sentiment'] < 0].index.tolist()

    context.long_weight, context.short_weight = my_compute_weights(context)

    
def my_compute_weights(context):

    # Calculer des pondérations cibles égales pour nos positions longues et nos positions courtes.
    if len(context.longs)==0:
        long_weight = 0
    else:
        long_weight = 0.5/len(context.longs)
    
    if len(context.shorts)==0:
        short_weight = 0
    else:
        short_weight = -0.5/len(context.shorts)
 
    return long_weight, short_weight


def my_rebalance(context, data):

    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            order_target_percent(security, 0)

    for security in context.longs:
        if data.can_trade(security):
            order_target_percent(security, context.long_weight)

    for security in context.shorts:
        if data.can_trade(security):
            order_target_percent(security, context.short_weight)
            
    print(context.something)